# Engenharia de Conhecimento 21/22

## Rede de Bayes (genérica) em python


<img src="files/imagens/Figura-1-Exemplo-de-Rede-Bayesiana-Mili-an-et-al-2010.png" alt="Drawing" style="width: 500px;"/> 

Comecemos por importar os ficheiros importantes

In [1]:
from probabilityPlus import *
from prettytable import *
from utils import print_table

### Rede de Bayes genérica em Python

Vamos redefinir os nós da rede com base na classe `BayesNode`, mas em que mantemos a ideia que se colocarmos apenas um valor é o valor do `True`, derivamos o `False` através de $P(X=False)=1-P(X=True)$. Podemos no entanto ter variáveis não booleanas e construir distribuições de probabilidade para os casos não booleanos.
Criamos assim a classe `NoBayes`.

In [2]:
class NoBayes:
    """A conditional probability distribution for a boolean variable,
    P(X | parents). Part of a BayesNet."""

    def __init__(self, X, parents, cpt):
        """X is a variable name, 
          and parents a sequence of variable names or a space-separated string. 
          cpt, the conditional probability table, takes one of these forms:

        * A number, the unconditional probability P(X=true). You can
          use this form when there are no parents.

        * A dict {v: p, ...}, the conditional probability distribution
          P(X=true | parent=v) = p. When there's just one parent.
          p is a number int or floar or a probability distributions in a dictionary

        * A dict {(v1, v2, ...): p, ...}, the distribution P(X=true |
          parent1=v1, parent2=v2, ...) = p. Each key must have as many
          values as there are parents. You can use this form always;
          the first two are just conveniences.
          p is a number int or float or a probability distributions in a dictionary

        In all boolean cases the probability of X being false is left implicit,
        since it follows from P(X=true).

        >>> X = BayesNode('X', '', 0.2)
        >>> Y = BayesNode('Y', 'P', {T: 0.2, F: 0.7})
        >>> Z = BayesNode('Z', 'P Q',
        ...    {(T, T): 0.2, (T, F): 0.3, (F, T): 0.5, (F, F): 0.7})
         but it might also be non-boolean and we pass a probability distribution instead of a value
        >>> X = BayesNode('X', '', {'a':12,'b':34,'c':30})
        >>> Y = BayesNode('Y', 'P', {T: {'a':12,'b':4,'c':3}, F: {'a':120,'b':34,'c':300}})
        >>> Z = BayesNode('Z', 'P Q',
        ...    {(T, T): 0.2, (T, F): 0.3, (F, T): 0.5, (F, F): 0.7})
         
        
        
        """

        
        
        if isinstance(parents, str):
            parents = parents.split()
        #print(parents)
        if parents==[]:
            # We store the table always in the third form above.
            if isinstance(cpt, (float, int)):  # no parents, 0-tuple, Boolean 
                cpt = {(): cpt}
            elif isinstance(cpt,dict):
                cpt = {():ProbDist(freq=cpt)}  # no parents, 0-tuple, non-boolean
        elif len(parents) == 1 and isinstance(cpt, dict):
            if not isinstance(list(cpt.keys())[0],tuple):
                cpt = {(v,): self.trata(pr) for v, pr in cpt.items()}
        else:
            cpt = {v: self.trata(pr) for v, pr in cpt.items()}
        #assert isinstance(cpt, dict)
        #for vs, p in cpt.items():
        #    print('uiui:',vs)
        #    assert isinstance(vs, tuple) and len(vs) == len(parents)
        #    assert all(isinstance(v, bool) for v in vs)
        #    assert 0 <= p <= 1

        self.variable = X
        self.parents = parents
        self.cpt = cpt
        vals=list(cpt.values())[0]
        #print('Os vals:',vals)
        if isinstance(vals,(float,int)):
            self.domain = [True,False]
        else:
            self.domain = list(vals.prob.keys())
        self.children = []
        
    def trata(self,p):
        if isinstance(p,(float,int)):
            return p
        else:
            return ProbDist(freq=p)

    def p(self, value, event):
        """Return the conditional probability
        P(X=value | parents=parent_values), where parent_values
        are the values of parents in event. (event must assign each
        parent a value.)
        >>> bn = BayesNode('X', 'Burglary', {T: 0.2, F: 0.625})
        >>> bn.p(False, {'Burglary': False, 'Earthquake': True})
        0.375
        """
        if isinstance(value, bool):
            ptrue = self.cpt[event_values(event, self.parents)]
            return ptrue if value else 1 - ptrue
        else:
            return self.cpt[event_values(event, self.parents)].prob[value]

    def sample(self, event):
        """Sample from the distribution for this variable conditioned
        on event's values for parent_variables. That is, return True/False
        at random according with the conditional probability given the
        parents."""
        return probability(self.p(True, event))

    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))

Vamos alterar ligeiramente a classe `BayesNet`, a que chamaremos de `RedeBayes`.

In [3]:
class RedeBayes:
    """Bayesian network containing only boolean-variable nodes."""

    def __init__(self, node_specs=None):
        """Nodes must be ordered with parents before children."""
        self.nodes = []
        self.variables = []
        node_specs = node_specs or []
        for node_spec in node_specs:
            self.add(node_spec)

    def add(self, node_spec):
        """Add a node to the net. Its parents must already be in the
        net, and its variable must not."""
        node = NoBayes(*node_spec)
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        self.nodes.append(node)
        self.variables.append(node.variable)
        for parent in node.parents:
            self.variable_node(parent).children.append(node.variable)

    def variable_node(self, var):
        """Return the node for the variable named var.
        >>> burglary.variable_node('Burglary').variable
        'Burglary'"""
        for n in self.nodes:
            if n.variable == var:
                return n
        raise Exception("No such variable: {}".format(var))

    def variable_values(self, var):
        """Return the domain of var. change this please"""
        node=self.variable_node(var)
        return node.domain

    def __repr__(self):
        return 'BayesNet({0!r})'.format(self.nodes)

#### Exemplo de teste da variação do exemplo standard Burglary
Para testar já as novas classes, vamos modelizar de um modo diferente o exemplo standard do assalto, em que consideramos que a variável `Earthquake` deixa de ser booleana (há ou não terramoto) e passa a ter 3 nuances: `strong`, `light` e `none`.

In [4]:
T, F = True, False

rb_burglary = RedeBayes([
    ('Burglary', '', 0.001),
    ('Earthquake', '', {"strong":1,"light":10,"none":89 }),
    ('Alarm', 'Burglary Earthquake',
          {(T, 'strong'): 0.95, (T, 'light'): 0.80, (T, 'none'): 0.76, (F, 'strong'): 0.8, (F, 'light'): 0.3, (F, 'none'): 0.001}),
    ('JohnCalls', 'Alarm', {T: 0.90, F: 0.05}),
    ('MaryCalls', 'Alarm', {T: 0.70, F: 0.01})
])
rb_burglary

BayesNet([('Burglary', ''), ('Earthquake', ''), ('Alarm', 'Burglary Earthquake'), ('JohnCalls', 'Alarm'), ('MaryCalls', 'Alarm')])

Vamos então aceder à tabela de probabilidade condicional para a variável `Earthquake`. É uma variável sem pais com 3 valores: strong, light and none.

In [5]:
rb_burglary.variable_node('Earthquake').cpt[()].prob

{'strong': 0.01, 'light': 0.1, 'none': 0.89}

Acedamos a $P(Earthquake=light)$

In [6]:
rb_burglary.variable_node('Earthquake').p('light',{})

0.1

Vamos aceder à probabilidade $P(Alarm=true)|Earthquake=none,Burglary=true)$

In [7]:
rb_burglary.variable_node('Alarm').p(T,{'Earthquake':'none','Burglary':T})

0.76

Eis uma função para mostrar a tabela de probabilidade conjunta.

In [8]:
# Função retirada da PL5
def display(j):
    """
    Pretty display of joint distribution
    """
    pretty=PrettyTable()
    aux = j.variables.copy()
    aux.append('Prob')
    pretty.field_names = aux 
    for i in list(j.prob.keys()):
        pretty.add_row(i+(j[i],))
    print(pretty)

Afunção joint_distribution devolve a conjunta correspondente a uma Rede de Bayes de input. Vamos gerar a conjunta de burglary e depois façamos o seu display.

In [9]:
#joint_distribution(burglary)
joint_burglary = joint_distribution(rb_burglary)
display(joint_burglary)

+----------+------------+-------+-----------+-----------+------------------------+
| Burglary | Earthquake | Alarm | JohnCalls | MaryCalls |          Prob          |
+----------+------------+-------+-----------+-----------+------------------------+
|   True   |   strong   |  True |    True   |    True   |       5.985e-06        |
|   True   |   strong   |  True |    True   |   False   | 2.565000000000001e-06  |
|   True   |   strong   |  True |   False   |    True   | 6.649999999999998e-07  |
|   True   |   strong   |  True |   False   |   False   | 2.8499999999999997e-07 |
|   True   |   strong   | False |    True   |    True   | 2.5000000000000027e-10 |
|   True   |   strong   | False |    True   |   False   | 2.4750000000000026e-08 |
|   True   |   strong   | False |   False   |    True   | 4.750000000000005e-09  |
|   True   |   strong   | False |   False   |   False   | 4.702500000000005e-07  |
|   True   |   light    |  True |    True   |    True   |        5.04e-05        |
|   

#### Exemplo de teste 2, o exemplo da Gripe
Sabemos que a probabilidade a priori de uma pessoa estar vacinada é de 1 em 1000.
Sabemos também que a probabilidade de uma pessoa apanhar gripe quando está vacinada (2 em 1000) é 10 vezes menor do que quando não está vacinada. 
Por outro lado, sabemos que se uma pessoa tiver gripe as brobabilidades de não ter febere nenhuma, de ter febre moderada e de ter febre elevada são respectivamente de 25%, 25% e 50%. Mas se não tiver gripe os valores serão respectivamente de: 97%, 2% e 1%.

Construamos a Rede de Bayes que corresponde a este domínio sabendo que teremos 4 variáveis:

Vacinada: uma pessoa está ou não vacinada - com domíno {true,false}.
Gripe: uma pessoa sofre ou não de gripe - com domíno {true,false}.
Febre: uma pessoa está sem febre, com febre moderada ou febre elevada - com domíno {não,moderada,elevada}.
DoresCabeca: uma pessoa tem ou não dores de cabeça - com domíno {true,false}.

In [10]:
rb_gripe = RedeBayes([
            ('Vacinada', '', 0.001),
            ('Gripe', 'Vacinada', {T: 0.002, F: 0.02}),
            ('Febre', 'Gripe', {T:{'não':25, 'moderada':25, 'elevada':50},F:{'não':97, 'moderada':2,  'elevada':1}}),
            ('DoresCabeca', 'Gripe', {T: 0.5,   F: 0.03})])

Vamos ler o valor de $P(DoresCabeca=true)$:

In [11]:
rb_gripe.variable_node('DoresCabeca').cpt

{(True,): 0.5, (False,): 0.03}

Vamos ler o valor de $P(Febre=elevada|Gripe=true)$:

In [12]:
rb_gripe.variable_node('Febre').p('elevada',{'Gripe':T})

0.5

E também o valor de $P(Vacinada=true)$:

In [13]:
rb_gripe.variable_node('Vacinada').domain

[True, False]

Eis uma função para fazer o display de uma cpt (conditional probability table, in english):

In [14]:
def display_cpt(node):
    header='P(' + node.variable + '|' + ','.join(node.parents) +'):'
    print(header)
    pretty=PrettyTable()
    aux = node.parents.copy()
    aux.append(node.variable)
    aux.append("Prob")
    pretty.field_names = aux 
    #print(pretty.field_names)   # when it is boolean
    if node.domain==[True,False]:
        for i in list(node.cpt.keys()):
            #print(i)
            pretty.add_row(i+('True',)+(str(node.cpt[i]),))
            pretty.add_row(i+('False',)+(str(1-node.cpt[i]),))
        print(pretty)
    else:
        for i in list(node.cpt.keys()):
            for j in node.domain:
                pretty.add_row(i+(j,)+(str(node.cpt[i].prob[j]),))

        print(pretty)

Façamos o display da cpt associada à variável Febre, i.e. $P(Febre|Gripe)$:

In [15]:
display_cpt(rb_gripe.variable_node('Febre'))

P(Febre|Gripe):
+-------+----------+------+
| Gripe |  Febre   | Prob |
+-------+----------+------+
|  True |   não    | 0.25 |
|  True | moderada | 0.25 |
|  True | elevada  | 0.5  |
| False |   não    | 0.97 |
| False | moderada | 0.02 |
| False | elevada  | 0.01 |
+-------+----------+------+


E agora apresentemos a tabela com a conjunta completa (envolvendo a conjunção de todas as variáveis) para este problema:

In [16]:
distribution_flu = joint_distribution(rb_gripe)
display(distribution_flu)

+----------+-------+----------+-------------+------------------------+
| Vacinada | Gripe |  Febre   | DoresCabeca |          Prob          |
+----------+-------+----------+-------------+------------------------+
|   True   |  True |   não    |     True    |        2.5e-07         |
|   True   |  True |   não    |    False    |        2.5e-07         |
|   True   |  True | moderada |     True    |        2.5e-07         |
|   True   |  True | moderada |    False    |        2.5e-07         |
|   True   |  True | elevada  |     True    |         5e-07          |
|   True   |  True | elevada  |    False    |         5e-07          |
|   True   | False |   não    |     True    | 2.9041799999999997e-05 |
|   True   | False |   não    |    False    |      0.0009390182      |
|   True   | False | moderada |     True    |       5.988e-07        |
|   True   | False | moderada |    False    | 1.9361199999999997e-05 |
|   True   | False | elevada  |     True    |       2.994e-07        |
|   Tr

#### Exemplo de teste 3:

<img src="imagens\RedeBayesLimitesVelocidade.PNG" alt="Drawing" style="width: 550px;"/>

que é modelizado com esta rede de Bayes:

<img src="imagens\ResRedeBayesLimitesVelocidade.PNG" alt="Drawing" style="width: 650px;"/>

sendo:

    P a variável correspondente a condutor com pressa (True,False)
    V a variável correspondente à velocidade (v1, v2, v3)
        v1 corresponde a velocidade < 120 km/h
        v2 corresponde a velocidade entre 120km/h e 140km/h: [120,140]
        v3 corresponde a velocidade > 140 km/h
    M a variável correspondente a apanhar mult (True,False)
    R a varável associada a ser detectado pelo radar (True,False)

Como não se indica a probabilidade de os condutores terem pressa, vamos arranjar um valor para completar a rede: por exemplo, 60% dos condutores têm pressa.

In [17]:
rb_on_the_road = RedeBayes([
            ('P', '', 0.60),
            ('R', '', 0.20),
            ('V', 'P', {T:{'v1':0.2, 'v2':0.5, 'v3':0.3},F:{'v1':0.6, 'v2':0.3,'v3':0.1}}),
            ('M', 'V R', {('v1',T):0,('v2',T):0.6,('v3',T):0.9,('v1',F):0,('v2',F):0,('v3',F):0})])

In [18]:
rb_on_the_road.variable_node('M').cpt

{('v1', True): 0,
 ('v2', True): 0.6,
 ('v3', True): 0.9,
 ('v1', False): 0,
 ('v2', False): 0,
 ('v3', False): 0}

In [19]:
display_cpt(rb_on_the_road.variable_node('M'))

P(M|V,R):
+----+-------+-------+---------------------+
| V  |   R   |   M   |         Prob        |
+----+-------+-------+---------------------+
| v1 |  True |  True |          0          |
| v1 |  True | False |          1          |
| v2 |  True |  True |         0.6         |
| v2 |  True | False |         0.4         |
| v3 |  True |  True |         0.9         |
| v3 |  True | False | 0.09999999999999998 |
| v1 | False |  True |          0          |
| v1 | False | False |          1          |
| v2 | False |  True |          0          |
| v2 | False | False |          1          |
| v3 | False |  True |          0          |
| v3 | False | False |          1          |
+----+-------+-------+---------------------+


In [20]:
rb_on_the_road.variable_node('M').domain

[True, False]

Revistemos a função `enumeration_ask`que faz uso da função `enumeration_all`: 

In [21]:
def enumeration_ask(X, e, bn):
    """
    [Figure 14.9]
    Return the conditional probability distribution of variable X
    given evidence e, from BayesNet bn.
    """
    assert X not in e, "Query variable must be distinct from evidence"
    Q = ProbDist(X)
    for xi in bn.variable_values(X):
        Q[xi] = enumerate_all(bn.variables, extend(e, X, xi), bn)
    return Q.normalize()


def enumerate_all(variables, e, bn):
    """Return the sum of those entries in P(variables | e{others})
    consistent with e, where P is the joint distribution represented
    by bn, and e{others} means e restricted to bn's other variables
    (the ones other than variables). Parents must precede children in variables."""
    if not variables:
        return 1.0
    Y, rest = variables[0], variables[1:]
    Ynode = bn.variable_node(Y)
    if Y in e:
        return Ynode.p(e[Y], e) * enumerate_all(rest, e, bn)
    else:
        return sum(Ynode.p(y, e) * enumerate_all(rest, extend(e, Y, y), bn)
                   for y in bn.variable_values(Y))

Voltemos à rede burglary e perguntemos qual a distribuição deJohn probabilidade de o João telefonar

In [22]:
enumeration_ask('JohnCalls',{},rb_burglary).prob

{True: 0.0836744585, False: 0.9163255414999999}

E se quisermos saber qual a probabilidade de haver um assalto se o João telefonar:

In [23]:
enumeration_ask('Burglary',{'JohnCalls':True},rb_burglary).prob

{True: 0.008377885110544218, False: 0.9916221148894558}

E qual a distribuição de probabilidade condicional de `Alarm` dado que ambos telefonaram, Maria e João:

In [24]:
enumeration_ask('Alarm',{'JohnCalls':True,'MaryCalls':True},rb_burglary).prob

{True: 0.9811237377457541, False: 0.01887626225424589}

E agora queremos saber qual a crença mais provável para uma determinada variável dada uma certa evidência.

In [25]:
#  a crença mais provável (refazer para lidar com empates)
def keywithmaxval(d):
    """ a) create a list of the dict's keys and values; 
        b) return the key with the max value"""  
    v=list(d.values())
    vmax=max(v)
    max_keys=[k for k in list(d.keys()) if d[k]==vmax]
    random.shuffle(max_keys)
    return max_keys[0]

def most_probable_belief(v,e,rede):
    dist_fantasma=enumeration_ask(v, e, rede).prob
    #print(dist_fantasma)
    return keywithmaxval(dist_fantasma)

E agora qual o valor de `Alarm` mais provável dado que ambos telefonaram?

In [26]:
most_probable_belief('Alarm',{'JohnCalls':True,'MaryCalls':True},rb_burglary)

True

## Variáveis com elementos no domínio que são tuplos
Notem que as `cpts` das Redes de Bayes na classe `BayesNode` são representadas por um dicionário e que o formato das chaves correspondem a tuplos com os valores das variáveis dos nós pais. Assim, existirá necessariamente um conflito se representarem os valores de uma variável com valores no seu domínio correspondentes, por exemplo, a tuplos com coordenadas $(L,C)$. Por exemplo, o tuplo $(1,1)$ na chave da cpt dessa variável, por exemplo, será interpretado como uma entrada de uma tabela com duas variáveis pais, ambas com o valor $1$.
Assim, nesse caso, o domínio de uma variável $Posicao$ que indica a posição numa grelha dimensional, não deve ser formado por duplos mas strings com duplos:

$Dom(Posicao)=\{\ '(1,1)\ ',\ '(1,2)\ ',\ '(1,3)\ ',\ '(1,4)\ ',\ '(2,1)\ ',\ '(2,2)\ ',\ '(2,3)\ ',\ '(2,4)\ ',\ '(3,1)\ ',(3,2)\ ',(3,3)\ ',\ '(3,4)\ ',\ '(4,1)\ ',\ '(4,2)\ ',\ '(4,3)\ ',\ '(4,4)\ '\}$